In [1]:
from pycalphad import Database, Model
from pycalphad.core.utils import make_callable
import pycalphad.variables as v
import numba
import numpy as np

In [2]:
db_alpt = Database('alpt_2015.tdb')
my_phases_alpt = ['PT2AL', 'PT5AL21', 'PT5AL3', 'LIQUID', 'AL3NI2', 'PT8AL21', 'PT2AL3', 'FCC_L12', 'BCC_B2']
mod = Model(db_alpt, ['AL', 'PT', 'VA'], 'LIQUID', parameters={'VX45': 0., 'VX46': 0., 'VX47': 0.})

In [29]:
from sympy import lambdify
from sympy.printing.lambdarepr import LambdaPrinter
import autograd.numpy as anp
func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules='numpy', printer=LambdaPrinter)

vec_func = numba.vectorize([numba.float64(numba.float64, numba.float64, numba.float64)],
                           nopython=False, target='parallel')(func)
old_func = make_callable(mod.ast, mod.variables)

x1 = np.random.rand(100, 100,1)
x2 = np.random.rand(100, 100,5000)
x3 = np.random.rand(100, 100,5000)
np.testing.assert_allclose(vec_func(x1, x2, x3), old_func(x1, x2, x3))
%timeit vec_func(x1, x2, x3)
%timeit old_func(x1, x2, x3)

1 loops, best of 3: 3.15 s per loop
1 loops, best of 3: 13.8 s per loop
